# 1. 讀取檔案

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

# 讀取訓練與測試資料
df = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# 觀察資料規模與前五筆資料
print(f"Train shape: {df.shape}, Test shape: {df_test.shape}")
df.head()

Train shape: (5634, 21), Test shape: (1409, 20)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4950-BDEUX,Male,0,No,No,35,No,No phone service,DSL,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,49.20,1701.65,No
1,7993-NQLJE,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,75.10,1151.55,No
2,7321-ZNSLA,Male,0,Yes,Yes,13,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,40.55,590.35,No
3,4922-CVPDX,Female,0,Yes,No,26,Yes,No,DSL,No,...,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),73.50,1905.70,No
4,2903-YYTBW,Male,0,Yes,Yes,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,44.55,44.55,No


# 2.資料預處理

In [32]:
# HINT: 原始資料的 'TotalCharges' 含有空白字串 " "，這會導致模型報錯。
# 將目標欄位 Churn (Yes/No) 轉換為數值 (1/0)
target = 'Churn'
df[target] = df[target].apply(lambda x: 1 if x == 'Yes' else 0)

# 3.特徵工程

In [33]:
# 定義要使用的特徵欄位
# HINT：這裡只選用了數值特徵。試著加入其他特徵（ex:文字欄位等），
# 或者加入 K-Means 分群結果作為新特徵，分數通常會提升。
features = ['MonthlyCharges', 'SeniorCitizen']

# 準備訓練用的 X 和 y
X = df[features]
y = df[target]

# 4. 訓練模型

In [34]:
# 切分訓練集與驗證集 (Validation Set)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("開始訓練模型...")

# 建立並訓練 Decision Tree 模型
# HINT：Decision Tree 只是基礎模型 (Baseline)。
# 若想獲得更高分，可以嘗試調整參數 (如 max_depth)，或是改用 XGBoost等模型。
# 另外，針對資料不平衡 (Imbalance) 的問題，也可以嘗試處理。
model = DecisionTreeClassifier(max_depth=4, random_state=42)
model.fit(X_train, y_train)

# 進行預測 (輸出機率值)
y_pred_proba = model.predict_proba(X_val)[:, 1]

# 計算並印出本地驗證分數
val_auc = roc_auc_score(y_val, y_pred_proba)
print(f"模型訓練完成！")
print(f"本地驗證分數 (Validation AUC): {val_auc:.4f}")

開始訓練模型...
模型訓練完成！
本地驗證分數 (Validation AUC): 0.6518


# 5.輸出提交檔案

In [35]:
# 準備測試集的特徵 (需與訓練集一致)
X_test = df_test[features]
X_test = X_test.fillna(0) 

# 使用訓練好的模型對測試集進行預測
predictions = model.predict_proba(X_test)[:, 1]

# 建立提交檔案
submission = pd.DataFrame({'customerID': df_test['customerID'], 'Churn': predictions})

# 存檔
submission.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：__________ 學號：__________

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：__________  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：__________________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：__________________________________________________________________  
    該次實驗分數： ____________  
【實驗 2】  
    我做的修改：__________________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：__________________________________________________________________  
    該次實驗分數： ____________  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：______________________________________________    